<a href="https://colab.research.google.com/github/agentescrossmodais/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd
from numba import jit, prange
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from google.colab import drive
import urllib.request

# Montar Google Drive
drive.mount('/content/drive')

# Configurações globais
DRIVE_MOUNT_PATH = "/content/drive"
GRAPHPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive/graficos")
SUMMARYPATH = os.path.join(DRIVE_MOUNT_PATH, "MyDrive/sumarios")
PALETTE = sns.color_palette("bright")
FIGSIZE = (10, 6)
CSV_URL = "https://docs.google.com/spreadsheets/d/1FA_j6pAdUXh0q-2WUOiS-JeVU2rAhxaQExroDvnQPoM/gviz/tq?tqx=out:csv&gid=1263817676"

# Função para criar diretórios
def create_directories(directories):
    for directory in directories:
        os.makedirs(directory, exist_ok=True)

# Função para carregar dados
def load_data(url, delimiter=',', encoding='utf-8'):
    try:
        df = pd.read_csv(url, delimiter=delimiter, encoding=encoding)
        return dd.from_pandas(df, npartitions=4)
    except Exception as e:
        print(f"Erro ao carregar dados: {e}")
        return None

# Função para salvar gráfico
def save_fig(filename, graphpath):
    plt.savefig(os.path.join(graphpath, filename), dpi=300, bbox_inches='tight')
    plt.close()

# Função para agrupar dados
def group_data(df):
    df_computed = df.compute()
    df_computed['CargaHorariaGrupo'] = pd.qcut(df_computed['CargaHorariaTotal'], 4, labels=["Baixa", "Média Baixa", "Média Alta", "Alta"])
    return dd.from_pandas(df_computed, npartitions=4)

# Classe para gerar gráficos
class GraphGenerator:
    def __init__(self, df, graphpath):
        self.df = df
        self.graphpath = graphpath

    def _adjust_palette(self, hue):
        n_unique = self.df[hue].nunique().compute()
        return sns.color_palette("bright", n_unique) if n_unique > len(PALETTE) else PALETTE

    def _generate_graph(self, kind, x, y=None, hue=None, title=None, filename=None, split=False, bins=20):
        try:
            plt.figure(figsize=FIGSIZE, facecolor='black')
            df_computed = self.df.compute()

            kwargs = {'data': df_computed, 'x': x, 'hue': hue, 'palette': self._adjust_palette(hue)}
            if y:
                kwargs['y'] = y
            if split:
                kwargs['split'] = True
            if kind == 'histplot':
                kwargs['bins'] = bins
                kwargs['kde'] = True

            getattr(sns, kind)(**kwargs)
            plt.gca().set_facecolor('black')
            plt.title(title, color='white')
            save_fig(filename, self.graphpath)

        except Exception as e:
            print(f"Erro ao gerar gráfico {filename}: {e}")

    def generate_violinplot(self, **kwargs):
        self._generate_graph(kind='violinplot', **kwargs)

    def generate_histplot(self, **kwargs):
        self._generate_graph(kind='histplot', **kwargs)

    def generate_boxplot(self, **kwargs):
        self._generate_graph(kind='boxplot', **kwargs)

    def generate_lineplot(self, **kwargs):
        self._generate_graph(kind='lineplot', **kwargs)

    def generate_kdeplot(self, **kwargs):
        self._generate_graph(kind='kdeplot', **kwargs)

    def generate_barplot(self, **kwargs):  # Função para gráfico de barras
        self._generate_graph(kind='barplot', **kwargs)

    def generate_polynomial_regression_plot(self, x_col, y_col, degree=3, title=None, filename=None):
        try:
            df_computed = self.df.compute()
            x = df_computed[x_col].values.reshape(-1, 1)
            y = df_computed[y_col].values

            # Criar uma regressão polinomial
            poly = PolynomialFeatures(degree=degree)
            x_poly = poly.fit_transform(x)
            poly_reg = LinearRegression()
            poly_reg.fit(x_poly, y)

            # Previsão usando o modelo polinomial
            y_pred = poly_reg.predict(x_poly)

            # Gráfico
            plt.figure(figsize=FIGSIZE, facecolor='black')
            plt.scatter(x, y, color='blue', label="Dados reais")
            plt.plot(x, y_pred, color='red', label=f"Regressão Polinomial (grau {degree})")
            plt.gca().set_facecolor('black')
            plt.title(title, color='white')
            plt.xlabel(x_col)
            plt.ylabel(y_col)
            plt.legend()

            save_fig(filename, self.graphpath)
        except Exception as e:
            print(f"Erro ao gerar gráfico {filename}: {e}")

# Função para gerar todos os gráficos
def generate_all_graphs(df, graph_generator):
    graph_specs = [
        {'type': 'violinplot', 'x': 'CargaHorariaGrupo', 'y': 'PublicoDiretoEstimado', 'hue': 'ObjetivosDeDesenvolvimentoSustentavel',
         'title': 'Violin: Público Estimado por Carga Horária e Objetivos de Desenvolvimento', 'filename': 'violin_publico_carga.png'},
        {'type': 'histplot', 'x': 'CargaHorariaTotal', 'hue': 'ObjetivosDeDesenvolvimentoSustentavel',
         'title': 'Histograma: Carga Horária por Objetivos de Desenvolvimento', 'filename': 'hist_carga_od.png'},
        {'type': 'violinplot', 'x': 'ObjetivosDeDesenvolvimentoSustentavel', 'y': 'CargaHorariaTotal', 'hue': 'CargaHorariaGrupo', 'split': True,
         'title': 'Violin: Carga Horária por Objetivos de Desenvolvimento e Grupo de Carga', 'filename': 'violin_publico_carga_total.png'},
        {'type': 'polynomial_regression_plot', 'x_col': 'CargaHorariaTotal', 'y_col': 'PublicoDiretoEstimado', 'degree': 3,
         'title': 'Regressão Polinomial: Carga Horária x Público Direto Estimado', 'filename': 'polynomial_regression_carga_publico.png'}
    ]

    for spec in graph_specs:
        plot_type = spec.pop('type')
        getattr(graph_generator, f'generate_{plot_type}')(**spec)

# Função para gerar sumário estatístico
def generate_statistical_summary(df, summarypath):
    try:
        numeric_columns = df.select_dtypes(include=[np.number]).columns
        summary = df[numeric_columns].describe().compute().T
        summary['variance'] = df[numeric_columns].var().compute()
        summary['skewness'] = [skew(df[col].compute()) for col in numeric_columns]
        summary['kurtosis'] = [kurtosis(df[col].compute()) for col in numeric_columns]
        summary.to_csv(os.path.join(summarypath, "summary_spss.csv"))
        print(f"Sumário estatístico salvo em: {summarypath}")
    except Exception as e:
        print(f"Erro ao gerar sumário estatístico: {e}")

# Função para bootstrap com Numba
@jit(nopython=True, parallel=True)
def bootstrap_jit(data, boots=1000):
    bs_data = np.empty(boots)
    for b in prange(boots):
        total = 0
        for s in range(data.shape[0]):
            total += data[np.random.randint(0, data.shape[0])]
        bs_data[b] = total / data.shape[0]
    return bs_data

# Função para calcular intervalos de confiança com bootstrap
def bootstrap_confidence_intervals(data, boots=1000, alpha=0.05):
    boot_means = bootstrap_jit(data, boots)
    lower = np.percentile(boot_means, 100 * alpha / 2)
    upper = np.percentile(boot_means, 100 * (1 - alpha / 2))
    return lower, upper

# Função para gerar gráficos adicionais
def generate_additional_graphs(df, graph_generator):
    additional_graph_specs = [
        {'type': 'kdeplot', 'x': 'CargaHorariaTotal', 'hue': 'ObjetivosDeDesenvolvimentoSustentavel',
         'title': 'KDE: Carga Horária por Objetivos de Desenvolvimento', 'filename': 'kde_carga_od.png'},
        {'type': 'boxplot', 'x': 'CargaHorariaGrupo', 'y': 'PublicoDiretoEstimado', 'hue': 'ObjetivosDeDesenvolvimentoSustentavel',
         'title': 'Boxplot: Público Estimado por Carga Horária e Objetivos de Desenvolvimento', 'filename': 'box_publico_carga_od.png'},
        {'type': 'violinplot', 'x': 'ObjetivosDeDesenvolvimentoSustentavel', 'y': 'CargaHorariaTotal', 'hue': 'CargaHorariaGrupo', 'split': True,
         'title': 'Violin: Carga Horária por Objetivos de Desenvolvimento e Grupo de Carga', 'filename': 'violin_carga_od_grupo.png'}
    ]

    for spec in additional_graph_specs:
        plot_type = spec.pop('type')
        getattr(graph_generator, f'generate_{plot_type}')(**spec)

# Execução principal
create_directories([GRAPHPATH, SUMMARYPATH])

df = load_data(CSV_URL)

if df is not None:
    df = group_data(df)
    graph_generator = GraphGenerator(df, GRAPHPATH)

    required_columns = ['CargaHorariaTotal', 'PublicoDiretoEstimado', 'ObjetivosDeDesenvolvimentoSustentavel']
    if all(col in df.columns for col in required_columns):
        generate_all_graphs(df, graph_generator)
        generate_statistical_summary(df, SUMMARYPATH)
        generate_additional_graphs(df, graph_generator)

        data = np.random.normal(0, 1, size=100)
        lower, upper = bootstrap_confidence_intervals(data, boots=1000)
        print(f"Intervalo de confiança via bootstrap: {lower:.4f}, {upper:.4f}")
    else:
        print(f"Colunas necessárias {required_columns} não estão presentes no dataframe.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-11-d9cddce15074>:74: UserWarning: The palette list has more values (10) than needed (4), which may not be intended.
  getattr(sns, kind)(**kwargs)


Sumário estatístico salvo em: /content/drive/MyDrive/sumarios


<ipython-input-11-d9cddce15074>:74: UserWarning: Dataset has 0 variance; skipping density estimate. Pass `warn_singular=False` to disable this warning.
  getattr(sns, kind)(**kwargs)
<ipython-input-11-d9cddce15074>:74: UserWarning: The palette list has more values (10) than needed (4), which may not be intended.
  getattr(sns, kind)(**kwargs)


Intervalo de confiança via bootstrap: -0.1006, 0.2247
